<a href="https://colab.research.google.com/github/YasmineSarraj/M3-Assignments-Deep-Learning/blob/main/Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Mar 13 16:54:48 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0    30W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:

!pip install gradio -q 

In [3]:
!pip uninstall whisper


In [4]:
!pip install openai-whisper==20230117

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import whisper

In [6]:

model = whisper.load_model("medium")

In [7]:
url = 'https://podcasts.google.com/feed/aHR0cHM6Ly9mZWVkcy5mZWVkYnVybmVyLmNvbS9PbG9naWVz/episode/MTUwNTYzZGItMWI1My00YzY2LWI5MjAtNDRiMzBhMTkzOTg0?sa=X&ved=0CAUQkfYCahcKEwiwjPT-sdn9AhUAAAAAHQAAAAAQCg'

In [8]:
import requests

In [9]:
content = requests.get(url)

In [10]:
from bs4 import BeautifulSoup
import re

In [11]:
soup = BeautifulSoup(content.text)



In [12]:

podcast_url = re.findall("(?P<url>\;https?://[^\s]+)", content.text)[0].split(';')[1]

In [13]:
podcast_url 

'https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEthanHawke_2020.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2Ff4599c45-837a-4244-a432-7fdf4148809e%2FEthanHawke_2020_VO_Intro.mp3'

In [14]:
!wget -O podcast.mp3 {podcast_url}

--2023-03-13 16:55:38--  https://backend.endpoints.firstory-709db.cloud.goog/play.mp3?url=https%3A%2F%2Fdts.podtrac.com%2Fredirect.mp3%2Fdownload.ted.com%2Ftalks%2FEthanHawke_2020.mp3%3Fapikey%3D172BB350-0207%26prx_url%3Dhttps%3A%2F%2Fchtbl.com%2Ftrack%2F48D18%2Fhttps%3A%2F%2Fdovetail.prxu.org%2F70%2Ff4599c45-837a-4244-a432-7fdf4148809e%2FEthanHawke_2020_VO_Intro.mp3
Resolving backend.endpoints.firstory-709db.cloud.goog (backend.endpoints.firstory-709db.cloud.goog)... 34.80.100.125
Connecting to backend.endpoints.firstory-709db.cloud.goog (backend.endpoints.firstory-709db.cloud.goog)|34.80.100.125|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://dts.podtrac.com/redirect.mp3/download.ted.com/talks/EthanHawke_2020.mp3?apikey=172BB350-0207&prx_url=https://chtbl.com/track/48D18/https://dovetail.prxu.org/70/f4599c45-837a-4244-a432-7fdf4148809e/EthanHawke_2020_VO_Intro.mp3 [following]
--2023-03-13 16:55:39--  https://dts.podtrac.com/redirect.mp3/download

In [15]:
result = model.transcribe('podcast.mp3')

In [16]:
from whisper.utils import write_vtt

In [17]:
with open('sub.vtt', "w") as txt:
  write_vtt(result["segments"], file=txt)

In [18]:
with open('sub.vtt', "r") as txt:
 print('\n'.join(txt.readlines()))

WEBVTT



00:00.000 --> 00:14.120

You're listening to TED Talks Daily. I'm Elise Hugh.



00:14.120 --> 00:19.440

Creativity is both awesome to see and kind of mysterious in how it comes about, but could



00:19.440 --> 00:25.240

it be more easily cultivated? In his Archive talk from TED 2020, the actor, writer, and



00:25.240 --> 00:30.660

director Ethan Hawke offers his ideas on the magic of creativity and how to make it come



00:30.660 --> 00:36.920

alive in ourselves and others. That's coming up.



00:36.920 --> 00:41.600

Support for TED Talks Daily is from Progressive, home of the Name Your Price tool. You can



00:41.600 --> 00:45.520

say how much you want to pay for car insurance, and they'll show you coverage options that



00:45.520 --> 00:49.940

fit your budget. It's easy to start a quote. Visit progressive.com to get started. Progressive



00:49.940 --> 00:53.080

Casualty Insurance Company and Affiliates, price and coverage match limited by state



00:53.0

In [ ]:
import gradio as gr 
import whisper
from whisper.utils import write_vtt
import requests
import os

def inference(link):
  content = requests.get(link)
  podcast_url = re.findall("(?P<url>\;https?://[^\s]+)", content.text)[0].split(';')[1]
  print(podcast_url)
  

  download = requests.get(podcast_url)

  with open('podcast.mp3', 'wb') as f:
    f.write(download.content)

  result = model.transcribe('podcast.mp3')

  with open('sub.vtt', "w") as txt:
    write_vtt(result["segments"], file=txt)

  return (result['text'], 'sub.vtt')

  


title="why listen when you can read"
description="Get Podcast Transcript"
block = gr.Blocks()

with block:
    gr.HTML(
        """     <center> 
                <h1>Just read I guess</h1>
                <img src = 'https://static01.nyt.com/images/2019/04/06/smarter-living/sl-better-listener-guide-slide-5UDQ/sl-better-listener-guide-slide-5UDQ-jumbo.jpg' width = '50%'></img>
                </center>
        """
    )
    with gr.Group():
        with gr.Box():
          
          link = gr.Textbox(label="Google Podcasts Link")

          with gr.Row().style(mobile_collapse=False, equal_height=True): 
              btn = gr.Button("Get PodScript 🪄")
          
          text = gr.Textbox(
              label="listening", 
              placeholder="PodScript Output",
              lines=5)
          
          transcription = gr.File()     
       
          
          btn.click(inference, inputs=[link], outputs=[text,transcription])

block.launch(debug=True)


/usr/local/lib/python3.9/dist-packages/gradio/layouts.py:75: UserWarning: mobile_collapse is no longer supported.
  warnings.warn("mobile_collapse is no longer supported.")


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

https://pdst.fm/e/stitcher.simplecastaudio.com/89060b63-e603-4297-b6e8-5a654e50244e/episodes/4b27f11f-0f66-4137-b607-314adfecda33/audio/128/default.mp3?aid=rss_feed&amp


Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/gradio/routes.py", line 384, in run_predict
    output = await app.get_blocks().process_api(
  File "/usr/local/lib/python3.9/dist-packages/gradio/blocks.py", line 1032, in process_api
    result = await self.call_function(
  File "/usr/local/lib/python3.9/dist-packages/gradio/blocks.py", line 844, in call_function
    prediction = await anyio.to_thread.run_sync(
  File "/usr/local/lib/python3.9/dist-packages/anyio/to_thread.py", line 31, in run_sync
    return await get_asynclib().run_sync_in_worker_thread(
  File "/usr/local/lib/python3.9/dist-packages/anyio/_backends/_asyncio.py", line 937, in run_sync_in_worker_thread
    return await future
  File "/usr/local/lib/python3.9/dist-packages/anyio/_backends/_asyncio.py", line 867, in run
    result = context.run(func, *args)
  File "<ipython-input-23-78c806fa5ff3>", line 18, in inference
    result = model.transcribe('podcast.mp3')
  File "/usr/local/lib/